In [216]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import time
import json


In [217]:
#Funkcja do otwarcia url i zamiana na jason
def get_json(url):

    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"}

    resp = requests.get(url,headers=headers)

    soup = BeautifulSoup(resp.text,'html.parser')
    data_tag = soup.find('script',{'id':'__NEXT_DATA__'}).text
    
    data = json.loads(data_tag)

    return data 

#Funkcja do ściągnięcia linków do stron 
def pages_to_scrape(pages, limit):
    Lista_url= []
    for page in range(pages):

        search_url = f'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa?page={page}&by=LATEST&direction=ASC&limit={limit}'
        search_data = get_json(search_url)

        for prop in search_data['props']['pageProps']['data']['searchAds']['items']:
            print(prop['title']) #prop has the property summary - not everything
            
            prop_url = 'https://www.otodom.pl/pl/oferta/'+prop['slug']
            print(prop_url)
            Lista_url.append(prop_url)
    return Lista_url
    print('skończone')



In [218]:
#Wywołanie funkcji
Otodom_links = pages_to_scrape(2,10)


Mieszkanie, 200 m², Warszawa
https://www.otodom.pl/pl/oferta/mieszkanie-200-m-warszawa-ID2f1sl
Mieszkanie, 113,80 m², Warszawa
https://www.otodom.pl/pl/oferta/mieszkanie-113-80-m-warszawa-ID2OsgW
GOTOWE 75m2 + poddasze 90m2 ul. Tawułkowa 7ZC
https://www.otodom.pl/pl/oferta/gotowe-75m2-poddasze-90m2-ul-tawulkowa-7zc-ID2VUZD
Bezpośrednio Mieszkanie bezczynszowe Jabłonna
https://www.otodom.pl/pl/oferta/bezposrednio-mieszkanie-bezczynszowe-jablonna-ID3aCLI
Cztery pokoje na Mokotowie przy parku.
https://www.otodom.pl/pl/oferta/cztery-pokoje-na-mokotowie-przy-parku-ID3ciQ6
Pola Mokotowskie! Apartament z ogrodem na dachu !
https://www.otodom.pl/pl/oferta/pola-mokotowskie-apartament-z-ogrodem-na-dachu-ID3fICS
Gotowy Apartament - 2 pokoje 38m2 z meblami
https://www.otodom.pl/pl/oferta/gotowy-apartament-2-pokoje-38m2-z-meblami-ID3hWDC
Jedyne takie dwa ogródki 30/15 m2 , tuż przy park
https://www.otodom.pl/pl/oferta/jedyne-takie-dwa-ogrodki-30-15-m2-tuz-przy-park-ID3yjyD
Apartament z Recepcją bli

In [219]:
#Ustawianie kluczy i tworzenie pustego słownika
klucze_target = ['Area', 'Build_year', 'Building_floors_num', 'Building_type', 'Building_ownership', 'Construction_status', 'City', 'City_id', 'Country', 'Extras_types', 'Floor_no', 'MarketType', 'OfferType', 'Price', 'Price_per_m', 'ProperType', 'Province', 'Rent', 'Rooms_num', 'Subregion', 'Windows_type', 'env', 'Equipment_types', 'Heating', 'Media_types', 'Security_types', 'Building_material', 'user_type']
klucze_ad = ['id', 'market', 'slug', 'advertType', 'createdAt', 'modifiedAt', 'description', 'exlusiveOffer']
klucze_owner = ['id', 'name', 'phones']
klucze_agency = ['id', 'name']
klucze_location_coordinates = ['latitude', 'longitude']
klucze_district = ['code', 'id', 'name']
wszystkie_mieszkanie = []

#Tworzenie petli, zeby przechodzilo, przez wszystki linki
for link in Otodom_links:
    search_data = get_json(link)
    mieszkanie = {}
    # Pętla dla kluczy w sekcji 'target'
    for klucz in klucze_target:
        mieszkanie[klucz] = search_data['props']['pageProps']['ad']['target'].get(klucz)

    # Pętla dla kluczy w sekcji 'ad'
    for klucz in klucze_ad:
        mieszkanie[klucz] = search_data['props']['pageProps']['ad'].get(klucz)

    # Pętla dla kluczy w sekcji 'owner'
    for klucz in klucze_owner:
        mieszkanie['owner_' + klucz] = search_data['props']['pageProps']['ad']['owner'].get(klucz)

    #Pętla dla kluczy w sekcji 'agency'
    if search_data['props']['pageProps']['ad']['agency'] is not None:
        for klucz in klucze_agency:
            mieszkanie['agency_' + klucz] = search_data['props']['pageProps']['ad']['agency'].get(klucz)
    

    # Pętla dla kluczy w sekcji 'coordinates'
    for klucz in klucze_location_coordinates:
        mieszkanie[klucz] = search_data['props']['pageProps']['ad']['location']['coordinates'].get(klucz)

   # #Pętla dla kluczy w sekcji 'district'
    if search_data['props']['pageProps']['ad']['agency'] is not None:
        for klucz in klucze_district:
            mieszkanie['district_'+ klucz] = search_data['props']['pageProps']['ad']['location']['address']['district'].get(klucz)
    wszystkie_mieszkanie.append(mieszkanie)
time.sleep(5)


In [220]:
df = pd.DataFrame(wszystkie_mieszkanie)

In [242]:
pd.set_option('display.max_columns', None)

Wyposażenie: meble, pralka, zmywarka, lodówka, kuchenka, piekarnik, telewizor
Zabezpieczenia: rolety antywłamaniowe, drzwi / okna antywłamaniowe, domofon / wideofon, monitoring / ochrona, system alarmowy, teren zamknięty
Media: internet, telewizja kablowa, telefon
Informacje dodatkowe: balkon, pom. użytkowe, garaż/miejsce parkingowe, piwnica, ogródek, taras, winda, dwupoziomowe, oddzielna kuchnia, klimatyzacja, tylko dla niepalących


In [243]:
df.iloc[:5,:]

,Area,Build_year,Building_floors_num,Building_type,Building_ownership,Construction_status,City,City_id,Country,Extras_types,Floor_no,MarketType,OfferType,Price,Price_per_m,ProperType,Province,Rent,Rooms_num,Subregion,Windows_type,env,Equipment_types,Heating,Media_types,Security_types,Building_material,user_type,id,market,slug,advertType,createdAt,modifiedAt,description,exlusiveOffer,owner_id,owner_name,owner_phones,agency_id,agency_name,latitude,longitude,district_code,district_id,district_name
0,200,1937,1,[house],[full_ownership],[to_renovation],warszawa,26,Polska,[basement],[floor_1],secondary,sprzedaz,2700000,13500,mieszkanie,mazowieckie,NaN,[8],powiat-warszawa,[wooden],production,None,[gas],None,[entryphone],[brick],agency,33133193,SECONDARY,mieszkanie-200-m-warszawa-ID2f1sl,AGENCY,2014-11-10T12:05:12+01:00,2023-07-25T16:10:55+02:00,<p>SASKA KĘPA - piętro domu - w kameralnym dom...,None,53131,Anna Derda,[+48226163059],53131.0,Magenda Nieruchomości Marek Masny,52.235924,21.047396,saska-kepa,847,Saska Kępa
1,113.8,1987,10,[block],[full_ownership],[ready_to_use],warszawa,26,Polska,"[balcony, basement, garage, lift, separate_kit...",[floor_10],secondary,sprzedaz,1479400,13000,mieszkanie,mazowieckie,NaN,[4],powiat-warszawa,None,production,[furniture],[urban],"[cable-television, internet]",None,[concrete_plate],agency,41577753,SECONDARY,mieszkanie-113-80-m-warszawa-ID2OsgW,AGENCY,2015-10-27T20:02:18+01:00,2023-07-06T21:30:06+02:00,"<p>*** WYSOKI STANDARD*** 4-pokojowe, wyposażo...",None,570652,Renata Burkacka,[+48667900040],570652.0,Loxton -Real Estate Group,52.141467,21.052604,natolin,743,Natolin
2,120,2022,1,[house],[full_ownership],[to_completion],warszawa,26,Polska,None,[ground_floor],primary,sprzedaz,730000,6083,mieszkanie,mazowieckie,NaN,[4],powiat-warszawa,[plastic],production,None,[gas],"[internet, phone]","[alarm, anti_burglary_door, closed_area, monit...",[brick],private,43594781,PRIMARY,gotowe-75m2-poddasze-90m2-ul-tawulkowa-7zc-ID2...,PRIVATE,2016-04-07T12:55:57+02:00,2023-07-31T12:40:37+02:00,<p><strong>MIESZKANIA BEZCZYNSZOWE – „OGRODY T...,None,1479135,MH Construction,[+48791660444],NaN,NaN,52.169532,21.182899,NaN,NaN,NaN
3,80,2022,2,[house],[full_ownership],[to_completion],jablonna,725,Polska,"[balcony, garage, garden, separate_kitchen]",[floor_1],primary,sprzedaz,635000,7938,mieszkanie,mazowieckie,NaN,[3],powiat-legionowski,[plastic],production,None,[gas],"[cable-television, internet, phone]","[closed_area, entryphone]",[cellular_concrete],private,46861318,PRIMARY,bezposrednio-mieszkanie-bezczynszowe-jablonna-...,PRIVATE,2017-01-19T20:42:33+01:00,2023-07-16T11:22:28+02:00,<p>Zapraszam do zapoznania się z ofertą kupna ...,None,2294756,Przemysław Walkowski,[+48500308808],NaN,NaN,52.363339,20.936792,NaN,NaN,NaN
4,97,2009,6,[apartment],None,None,warszawa,26,Polska,"[balcony, basement, garage, lift, separate_kit...",[floor_2],secondary,sprzedaz,2000000,20619,mieszkanie,mazowieckie,1800.0,[4],powiat-warszawa,[aluminium],production,None,[urban],None,None,None,agency,47261366,SECONDARY,cztery-pokoje-na-mokotowie-przy-parku-ID3ciQ6,AGENCY,2017-02-20T22:56:32+01:00,2023-08-03T09:49:36+02:00,<p></p><p></p><p>Czteropokojowy apartament w k...,None,2355750,Patrycja Kowacka,[+48668195550],2355750.0,Mansion Nieruchomości,52.202970,21.032850,dolny-mokotow,10150,Dolny Mokotów
